In [2]:
reload_lamb()

Let us take a $n$-projective meaning to be an n-tuple, where for $n>1$, the value is of type $t$ (or the intension thereof).  Normal composition operations apply to the $n=1$ case, and conjunction is used to (monotonically) add extra meaning layers together as needed.  

Nothing so far specifies any interpretation of any of these layers, and a specific projective meaning system will provide this.

In [67]:
%%lamb
||doctor|| = L x_e : (Doctor(x), Human(x))
||tall|| = L x_e : Tall(x)
||john1|| = J_e

INFO (meta): Coerced guessed type t for 'Doctor_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type t for 'Human_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type t for 'Tall_t' into <e,t>, to match argument 'x_e'


$||\mathbf{\text{doctor}}||^{}_{\langle{}e,(t, t)\rangle{}} \:=\: \lambda{} x_{e} \: . \: ({Doctor}({x}_{e}), {Human}({x}_{e}))$<br />
$||\mathbf{\text{tall}}||^{}_{\langle{}e,t\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Tall}({x}_{e})$<br />
$||\mathbf{\text{john1}}||^{}_{e} \:=\: {J}_{e}$

In [63]:
%%lamb
||john2|| = (J_e, Salient(J_e))

INFO (meta): Coerced guessed type t for 'Salient_t' into <e,t>, to match argument 'J_e'


$||\mathbf{\text{john2}}||^{}_{(e, t)} \:=\: ({J}_{e}, {Salient}({J}_{e}))$

In [59]:
comb = te("Lambda f_<e,(t,t)> : Lambda x_(e,t) : (f(x[0])[0], f(x[0])[1] & x[1])")
comb

(λ f_<e,(t,t)>: (λ x_(e,t): ((f_<e,(t,t)>((x_(e,t)[0_n]))[0_n]), ((f_<e,(t,t)>((x_(e,t)[0_n]))[1_n]) & (x_(e,t)[1_n])))))

In [64]:
comb(doctor.content)(john2.content).reduce_all()

(Doctor_<e,t>(J_e), (Human_<e,t>(J_e) & Salient_<e,t>(J_e)))

In [65]:
comb2 = te("Lambda f_<e,(t,t)> : Lambda x_e : (f(x)[0], f(x)[1])")
comb2

(λ f_<e,(t,t)>: (λ x_e: ((f_<e,(t,t)>(x_e)[0_n]), (f_<e,(t,t)>(x_e)[1_n]))))

In [66]:
comb2(doctor.content)(john1.content).reduce_all()

(Doctor_<e,t>(J_e), Human_<e,t>(J_e))

In [72]:
te("f_(e,e)[x_n]")

WARNING (meta): Using non-constant index; not well-supported at present.


(f_(e,e)[x_n])

In [104]:
def typedfun(typ):
    if isinstance(typ, str):
        typ = meta.tp(typ)
    if not typ.functional():
        raise types.TypeMismatch(typ, None, "Functional type needed for wrapped function")
    def dec(f):
        def do_typecheck(*args):
            ts = meta.get_type_system()
            if len(args) == 1:
                x = args[0]
            else:
                x = tuple(args)
            x = meta.TypedExpr.ensure_typed_expr(x, typ.left)
            result = f(x)
            result = meta.TypedExpr.ensure_typed_expr(result, typ.right)
            return result
        return do_typecheck
    return dec

In [109]:
@typedfun("<?,n>")
def test(e):
    return e.reduce_all()

In [112]:
test("(x_n, y_e)[0]")

x_n